In [1]:
import requests
from json import loads, dumps

# Basic Queries
Here's a simple query for the average CPU usage.

In [2]:
response = requests.get("http://127.0.0.1:3000/query", params={
    "query": dumps({
        "type": "basic",
        "epsilon": 1.0,
        "featurizer": "SELECT AVG(cpu_percent) FROM device_info.cpu_usage",
        "aggregator": "mean",
    })
})
qid1 = response.text

In [12]:
loads(requests.get("http://127.0.0.1:3000/query/%s" % qid1).text)

{'type': 'basic',
 'epsilon': 1.0,
 'featurizer': 'SELECT AVG(cpu_percent) FROM device_info.cpu_usage',
 'aggregator': 'mean',
 'id': 'eb9ca8d5-2387-43bc-96f9-f67e3d26effd',
 'status': 'COMPLETE',
 'count': 13,
 'result': 18.3}

We can take advantage of the full expressive power of SQL and compute slightly more complex statistics.

In [6]:
response = requests.get("http://127.0.0.1:3000/query", params={
    "query": dumps({
        "type": "basic",
        "epsilon": 1.0,
        "featurizer": "SELECT AVG(cpu_count*cpu_percent) FROM device_info.cpu_usage",
        "aggregator": "mean",
    })
})
qid3 = response.text

In [14]:
loads(requests.get("http://127.0.0.1:3000/query/%s" % qid3).text)

{'type': 'basic',
 'epsilon': 1.0,
 'featurizer': 'SELECT AVG(cpu_count*cpu_percent) FROM device_info.cpu_usage',
 'aggregator': 'mean',
 'id': '13ab5301-5137-4ff4-8eea-012396c5518e',
 'status': 'RUNNING',
 'count': 13,
 'result': 73.2}

# Generalized Queries
We can also compute the above value (cpu_count * cpu_percent) using a generalized query. Essentially, we'll move the heavy lifting over to Python (as opposed to SQL) and use a different mechanism to produce the differentially private result.

In [8]:
sampler = """
result = 0.0
for x in inputs:
    result += x["cpu_percent"] * x["cpu_count"]
outputs["x"] = result
""".strip()
response = requests.get("http://127.0.0.1:3000/query", params={
    "query": dumps({
        "type": "generalized",
        "epsilon": 1.0,
        "featurizer": "SELECT cpu_count, cpu_percent FROM device_info.cpu_usage",
        "sampler": sampler,
        "aggregator": "median",
    })
})
qid4 = response.text

In [15]:
loads(requests.get("http://127.0.0.1:3000/query/%s" % qid4).text)

{'type': 'generalized',
 'epsilon': 1.0,
 'featurizer': 'SELECT cpu_count, cpu_percent FROM device_info.cpu_usage',
 'sampler': 'result = 0.0\nfor x in inputs:\n    result += x["cpu_percent"] * x["cpu_count"]\noutputs["x"] = result',
 'aggregator': 'median',
 'id': 'd08861fe-5a1c-4bc1-b5a6-388b8aef9a0c',
 'status': 'COMPLETE',
 'count': 13,
 'result': 225.66549283241932}

This general approach can also be used to implement more sophisticated queries. For example, the following query fits a linear regression model to predict cpu usage from the timestamp - yes, this is obviously a somewhat farcical example - and returns the model coefficients.

In [10]:
sampler = """
import pandas as pd
import statsmodels.formula.api as sm

df = pd.DataFrame(inputs)
result = sm.ols(formula="cpu_percent ~ timestamp", data=df).fit()

outputs["x"] = result.params["timestamp"]
""".strip()
response = requests.get("http://127.0.0.1:3000/query", params={
    "query": dumps({
        "type": "generalized",
        "epsilon": 1.0,
        "featurizer": "SELECT timestamp, cpu_percent, cpu_count FROM device_info.cpu_usage",
        "sampler": sampler,
        "aggregator": "median",
    })
})
qid5 = response.text

In [16]:
loads(requests.get("http://127.0.0.1:3000/query/%s" % qid5).text)

{'type': 'generalized',
 'epsilon': 1.0,
 'featurizer': 'SELECT timestamp, cpu_percent, cpu_count FROM device_info.cpu_usage',
 'sampler': 'import pandas as pd\nimport statsmodels.formula.api as sm\n\ndf = pd.DataFrame(inputs)\nresult = sm.ols(formula="cpu_percent ~ timestamp", data=df).fit()\n\noutputs["x"] = result.params["timestamp"]',
 'aggregator': 'median',
 'id': 'd009f316-bb21-4952-ab81-a0e0e9b635eb',
 'status': 'COMPLETE',
 'count': 13,
 'result': 1.1401550791390784e-08}